In [16]:
from langchain_ollama import ChatOllama

llmMistral = ChatOllama(model="mistral7b-q4km:latest", temperature=0.2, base_url="http://localhost:11434")
llmMistralInstr = ChatOllama(model="mistral7b-instr-q4km:latest", temperature=0.2, base_url="http://localhost:11434")
llmLlama = ChatOllama(model="llama3_1-8b-instr-q4km:latest", temperature=0.2, base_url="http://localhost:11434")

In [17]:
from langchain_core.messages import SystemMessage, HumanMessage

human_prompt = """Generate a critical question, using this context: The claim is "Arizona officials caught changing ballots, have been arrested.".This is all the information know about the claim: The claim states that Arizona officials were caught changing ballots and have been arrested. It is a qualitative claim about a specific event, with clear geography (Arizona) but no source or methodological details and no explicit time frame, limiting its immediate checkability..These alerts point out omissions: [\'qualitative claim\', \'methodological details absent\', \'time period missing\'].Take into account previous questions asked: [\'From whose perspective is this claim being presented, and are there other viewpoints—such as statements from law‑enforcement agencies or the officials themselves—that might provide a more balanced understanding?\', \'If the claim were true, what legal and procedural consequences would follow, and how might those outcomes differ if the alleged actions were mischaracterized or occurred in a different context?\']."""


#responseMistral = llmMistral.invoke(human_prompt)
#responseMistralInstr = llmMistralInstr .invoke(human_prompt)
responseLlama = llmLlama.invoke(human_prompt)
#print(responseMistral.content) 
#print(responseMistralInstr.content) 
print(responseLlama.content) 

[[The claim is qualitative, so we cannot apply quantitative verification methods. Instead, we need to locate reliable sources—such as official statements or court records—that address the alleged ballot changes and arrests in Arizona..]]
[[This alert indicates that methodological details are absent, which limits our ability to assess the claim's credibility. We should seek out specific sources (e.g., police reports, court filings) rather than relying on vague or anecdotal information..]]
[[The missing time frame is a key gap in the claim's context. When did these alleged actions occur, and how might different temporal scenarios affect our interpretation of their significance?]]
[[This alert reminds us to consider whose perspectives are being highlighted or omitted when we evaluate this claim. Who stands to gain or lose if the story is accepted as true or false, and how does that shape the way we investigate it?]]
